In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import numpy as np

# Logistic model class
class LogisticModel:
    def __init__(self, coefficients):
        self.coefficients = coefficients

    def predict_mortality_probability(self, input_variables):
        input_with_intercept = np.array([1] + input_variables)
        linear_combination = np.dot(self.coefficients, input_with_intercept)
        probability = 1 / (1 + np.exp(-linear_combination))
        return probability

# Model coefficients (from Table II in Correccion.pdf)
model_coefficients = {
    1: [-8.562, 3.142],  # VR
    2: [-9.835, 3.227, 0.006],  # VR, Glucose
    3: [-11.115, 3.208, 0.01, 1.347],  # VR, Glucose, Covid(1)
    4: [-11.8, 3.376, 0.01, 1.374, 1.075],  # VR, Glucose, Covid(1), HAS(1)
    5: [-12.822, 3.33, 0.009, 1.821, 1.18, 0.002],  # VR, Glucose, Covid(1), HAS(1), DHL
    6: [-15.248, 3.107, 0.009, 0.002, 2.39, 0.047, 0.815],  # VR, Glucose, DHL, Covid(1), Age, Vasopressor(1)
    7: [-15.749, 3.377, 0.009, 1.83, 1.433, 0.002, -0.046, 0.008],  # VR, Glucose, Covid(1), HAS(1), DHL, Time, VT
    8: [-15.984, 3.303, 0.009, 1.902, 1.424, 0.002, -0.045, 0.008, 0.233],  # VR, Glucose, Covid(1), HAS(1), DHL, Time, VT, Creatinine
    9: [-14.981, 2.96, 0.009, 0.055, -0.048, 0.002, 2.701, -0.003, 0.223, 0.086],  # VR, Glucose, Age, Time, DHL, Covid(1), Platelets, Creatinine, PEEP
    10: [-15.784, 2.952, 0.009, 0.058, -0.046, 0.002, 2.729, -0.003, 0.212, 0.08, 0.021],  # VR, Glucose, Age, Time, DHL, Covid(1), Platelets, Creatinine, PEEP, BMI
}

# Variable names for each model
model_variable_names = {
    1: ["Ventilatory Ratio (VR)"],
    2: ["Ventilatory Ratio (VR)", "Glucose"],
    3: ["Ventilatory Ratio (VR)", "Glucose", "COVID status"],
    4: ["Ventilatory Ratio (VR)", "Glucose", "COVID status", "Hypertension (HAS)"],
    5: ["Ventilatory Ratio (VR)", "Glucose", "COVID status", "Hypertension (HAS)", "DHL"],
    6: ["Ventilatory Ratio (VR)", "Glucose", "DHL", "COVID status", "Age", "Vasopressor"],
    7: ["Ventilatory Ratio (VR)", "Glucose", "COVID status", "Hypertension (HAS)", "DHL", "Time", "Tidal Volume (VT)"],
    8: ["Ventilatory Ratio (VR)", "Glucose", "COVID status", "Hypertension (HAS)", "DHL", "Time", "Tidal Volume (VT)", "Creatinine"],
    9: ["Ventilatory Ratio (VR)", "Glucose", "Age", "Time", "DHL", "COVID status", "Platelets", "Creatinine", "PEEP"],
    10: ["Ventilatory Ratio (VR)", "Glucose", "Age", "Time", "DHL", "COVID status", "Platelets", "Creatinine", "PEEP", "BMI"]
}

# Function to calculate mortality probability
def calculate_mortality(model_number, input_variables):
    if model_number not in model_coefficients:
        raise ValueError(f"Model {model_number} is not available.")
    coefficients = model_coefficients[model_number]
    if len(input_variables) != len(coefficients) - 1:
        raise ValueError(f"Model {model_number} expects {len(coefficients)-1} input variables.")
    model = LogisticModel(coefficients)
    return model.predict_mortality_probability(input_variables)

# Function to handle the button click and calculate the probability
def on_calculate():
    try:
        # Get the selected model number
        model_number = int(model_dropdown.get())

        # Get input variable values from the text fields
        input_values = []
        for entry in input_entries:
            value = float(entry.get())
            input_values.append(value)

        # Calculate mortality probability
        probability = calculate_mortality(model_number, input_values)

        # Display the result in a message box
        messagebox.showinfo("Mortality Probability", f"Mortality Probability: {probability:.4f}")

    except Exception as e:
        messagebox.showerror("Error", str(e))

# Create the main application window
root = tk.Tk()
root.title("Mortality Probability Calculator")

# Dropdown menu for model selection
model_label = ttk.Label(root, text="Select Logistic Model:")
model_label.grid(column=0, row=0, padx=10, pady=10)
model_dropdown = ttk.Combobox(root, values=list(range(1, 11)))
model_dropdown.grid(column=1, row=0, padx=10, pady=10)
model_dropdown.current(0)

# Input fields for variables (this will be dynamically updated based on the selected model)
input_labels = []
input_entries = []

# Function to update input fields based on selected model
def update_input_fields(event):
    # Get the selected model number
    model_number = int(model_dropdown.get())
    
    # Clear existing input fields
    for label in input_labels:
        label.destroy()
    for entry in input_entries:
        entry.destroy()
    
    input_labels.clear()
    input_entries.clear()

    # Get the variable names for the selected model
    variable_names = model_variable_names[model_number]

    # Create input fields dynamically with proper variable names
    for i, var_name in enumerate(variable_names):
        input_label = ttk.Label(root, text=f"{var_name}:")
        input_label.grid(column=0, row=i + 1, padx=10, pady=5)
        input_entry = ttk.Entry(root)
        input_entry.grid(column=1, row=i + 1, padx=10, pady=5)
        input_labels.append(input_label)
        input_entries.append(input_entry)

# Bind the dropdown selection change to update the input fields
model_dropdown.bind("<<ComboboxSelected>>", update_input_fields)

# Button to calculate the probability
calculate_button = ttk.Button(root, text="Calculate Probability", command=on_calculate)
calculate_button.grid(column=0, row=11, columnspan=2, padx=10, pady=10)

# Start the application loop
root.mainloop()


In [11]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import numpy as np

# Logistic model class to predict mortality
class LogisticModel:
    def __init__(self, coefficients):
        self.coefficients = coefficients

    def predict_mortality_probability(self, input_variables):
        input_with_intercept = np.array([1] + input_variables)  # Add intercept term
        linear_combination = np.dot(self.coefficients, input_with_intercept)
        probability = 1 / (1 + np.exp(-linear_combination))  # Logistic function
        return probability

# Model coefficients for Tables VII to XV (including intercepts as first element)
model_coefficients = {
    '1 Variable': {
        1: [-8.562, 3.142],  # VR, Glucose
        2: [-1.291,0.429],  # VR, HAS
        3: [-1.193,0.001],  # VR, COVID-19
        4: [-0.171,-0.037],  # VR, Tidal Volume (VT)
        5: [-2.581,0.066],  # VR, Age
    },    
    '2 Variables': {
        1: [-9.835,3.227, 0.006],  # VR, Glucose
        2: [-9.161, 3.28, 1.093],  # VR, HAS
        3: [-8.978, 3.119, 0.779],  # VR, COVID-19
        4: [-11.171, 3.172, 0.006],  # VR, Tidal Volume (VT)
        5: [-10, 3.159, 0.03],  # VR, Age
    },
    '3 Variables': {
        1: [-10.153, 3.113, 0.718, 0.044],  # VR, COVID-19, Pres Plat
        2: [-10.412,3.374,0.006, 1.045],  # VR, Glucose, HAS
        3: [-12.758, 3.264, 0.007, 0.007],  # VR, Glucose, VT
        4: [-11.403, 3.147,0.737, 0.006],  # VR, COVID-19, VT
        5: [-9.321, 3.181, 0.007, -0.033],  # VR, Glucose, Time
    },
    '4 Variables': {
        1: [-12.012, 3.169, 1.742, 0.009, 0.001],  # VR, Glucose, COVID-19, HAS
        2: [-11.8, 3.376, 1.374, 1.075, 0.01],  # VR, COVID-19, Time, Glucose
        3: [-12.76, 3.237, 1.494, 0.009, 0.033],  # VR, Glucose, HAS, Time, VT
        4: [-11.426, 3.129,0.01, 1.534, 0.726],  # VR, COVID-19, VT, HAS, Glucose
        5: [-13.652,3.236, 1.295,0.006, 0.01],  # VR, Glucose, Age, Time, PEEP
    },
    '5 Variables': {
        1: [-14,72, 3.195, 0.002, 2.118, 0.008, 0.047],  # VR, Glucose, COVID-19, HAS, DHL
        2: [-12.822, 3.33, 1.821,1.18, 0.002,0.009],  # VR, Glucose, DHL, COVID-19, Age, Vasopressor
        3: [-12.487, 3.091, 0.002, 1.992, 0.009, 0.806],  # VR, Glucose, COVID-19, DHL, Time, Platelets
        4: [-14.251,3.417,1.344,1.153,0.007,0.01],  # VR, Glucose, COVID-19, Platelets, PEEP, BMI
        5: [-11.211,3.324,1.353,-0.039,1.193,0.01],  # VR, COVID-19, HAS, VT, DHL
    },
    '6 Variables': {
        1: [-14.403, 3.11, 0.009, -0.044, 0.002,0.053, 2.196],  # VR, Glucose, Time, DHL, Age, COVID-19
        2: [-15.248, 3.107, 0.002, 2.39, 0.009, 0.815, 0.047],  # VR, Glucose, DHL, COVID-19, Age, Vasopressor
        3: [-12.333,3.291,1.853,-0.045,1.341,0.002,0.009],  # VR, COVID-19, Time, HAS, DHL, Glucose
        4: [-16.085,3.405,1.806,1.266,0.007,0.002,0.009],  # VR, COVID-19, VT, HAS, DHL, Glucose
        5: [-15.018,3.339,0.002,2.138,0.009,0.968,0.039],  # VR, COVID-19, HAS, DHL, VT, Glucose
    },
    '7 Variables': {
        1: [-15.749,3.377,1.83,-0.046,1.433,0.008,0.002,0.009],  # VR, COVID-19, HAS, VT, Glucose, Time
        2: [-13.79, 3.075,0.009, -0.044, 0.002, 0.052, -0.004,2.556],  # VR, Glucose, Time, DHL, Age, Platelets
        3: [-15.578, 3.093, 0.009, -0.048, 0.002, 0.058, 0.088, 2.33],  # VR, Glucose, Time, DHL, Age, COVID-19
        4: [-15.528,3.252,0.002,2.403,0.009,0.888,0.748,0.04 ],  # VR, COVID-19, HAS, PEEP, Time
        5: [-14.343, 0.21, 3.023, 0.009, -0.043, 0.002, 0.05, 2.214],  # VR, COVID-19, Time, VT, HAS
    },
    '8 Variables': {
        1: [-15.984,3.303,1.902,-0.045,0.233,1.424,0.008,0.002,0.009],  # VR, COVID-19, HAS, VT, DHL, Glucose
        2: [-13.089,3.198,1.852,-0.046,0.233,1.274,0.027,0.002,0.009],  # VR, COVID-19, Pres Plat, HAS, DHL, Glucose
        3: [-13.745,0.21,2.987,0.009,-0.044,0.002,0.05,-0.004,2.593 ],  # VR, Glucose, Time, Platelets
        4: [-12.487,3.216,0.009,0.233,-0.044,1.905,0.002,1.309,0.001],  # Glucose, Age, Creatinine
        5: [-12.991,3.277,1.807,-0.047,1.202,0.453,0.002,0.026,0.009],  # VR, Time, etc...
    },
    '9 Variables': {
        1: [-16.224, 3.311,1.914,0.505,-0.047,0.196,1.314,0.008,0.002,0.009 ],  # Example for Table XIV (needs to be filled out)
        2: [-16.459,3.289,1.85,-0.047,0.232,1.386,0.008,0.002,0.023,0.009],  # VR, COVID-19, Pres Plat, HAS, DHL, Glucose
        3: [-14.981, 0.233, 2.96, 0.009, -0.048, 0.002, 0.055, -0.003, 0.086, 2.701],  # VR, Glucose, Time, Platelets
        4: [-15.969,3.303,0.009,0.233,-0.045,1.906,0.002,-0.001,1.426,0.008],  # Glucose, Age, Creatinine
        5: [-16.514,3.37,1.794,0.621,-0.048,1.262,0.008,0.002,0.021,0.009],  # VR, Time, etc...
    },
    '10 Variables': {
        1: [-16.681,3.295,1.863,0.502,-0.048,0.196,1.276,0.008,0.002,0.023,0.009],  # VR, COVID-19, DM, Creatinine, Time
        2: [-16.692,3.271,0.009,0.243,0.068,-0.048,1.966,0.002,-0.061,1.424,0.008],  # VR, COVID-19, Pres Plat, Creatinine, Glucose, DP
        3: [-15.784, 0.212, 2.952, 0.009, -0.046, 0.002, 0.058, -0.003, 0.08, 2.729, 0.021],  # VR, BMI, Glucose
        4: [-18.107,3.245,0.266,0.128,0.01,1.669,0.008,-0.049,-0.105,0.038,1.051],  # Glucose, Age, Creatinine
        5: [-16.783,3.12,0.002,2.538,0.017,0.195,0.009,0.853,0.731,0.01,0.039 ],  # VR, Time, etc...
    },
}

# Variable names for each model (Table VII to XV)
model_variable_names = {
    '1 Variable': {
        1: ["VR"],
        2: ["Creatinine"],
        3: ["DHL"],
        4: ["Time"],
        5: ["Pres_plat"],
    },
    '2 Variables': {
        1: ["VR","Glucose"],
        2: ["VR","HAS"],
        3: ["VR","Covid"],
        4: ["VR","VT"],
        5: ["VR","Age"],
    },
    '3 Variables': {
        1: ["VR","Covid","Pres_plat"],
        2: ["VR","Glucose","HAS"],
        3: ["VR","Glucose","VT" ],
        4: ["VR","Covid","VT" ],
        5: ["VR","Glucose","Time"],
    },
    '4 Variables': {
        1: ["VR","Covid","Glucose", "DHL"],
        2: ["VR","Covid","HAS", "Glucose"],
        3: ["VR","Covid","Glucose", "Age"],
        4: ["VR","Covid","Glucose", "Vasopressor"],
        5: ["VR","Covid","VT", "Glucose"],
    },
    '5 Variables': {
        1: ["VR","DHL","Covid", "Glucose","Age"],
        2: ["VR","Covid","HAS", "DHL","Glucose "],
        3: ["VR","DHL","Covid", "Glucose","Vasopressor"],
        4: ["VR","Covid","HAS", "VT","Glucose" ],
        5: ["VR","Covid","Time", "HAS","Glucose"],
    },
    '6 Variables': {
        1: ["VR,""Glucose","Time", "DHL","Age","Covid" ],
        2: ["VR","DHL","Covid", "Glucose","Vasopressor", "Age"],
        3: ["VR","Covid","Time", "HAS","DHL","Glucose" ],
        4: ["VR","Covid","HAS", "VT","DHL","Glucose"],
        5: ["VR","DHL","Covid", "Glucose","HAS","Age"],
    },
    '7 Variables': {
        1: ["VR","Covid","Time", "HAS","VT","DHL", "Glucose"],
        2: ["VR","Glucose", "Time","DHL","Age","Platelets", "Covid"],
        3: ["VR","Glucose","Time", "DHL","Age","PEEP", "Covid"],
        4: ["VR","DHL","Covid", "Glucose","HAS", "Vasopressor","Age"],
        5: ["Creatinine","VR","Glucose", "Time","DHL","Age", "Covid"],
    },
    '8 Variables': {
        1: ["VR","Covid", "Time","Creatinine","HAS", "VT","DHL","Glucose"],
        2: ["VR","Covid","Time", "Creatinine","HAS","Pres_plat", "DHL","Glucose"],
        3: ["Creatinine","VR","Glucose", "Time","DHL","Age", "Platelets","Covid"],
        4: ["VR","Glucose","Creatinine", "Time","Covid","DHL", "HAS","DP"],
        5: ["VR","Covid", "Time","HAS","DM", "DHL","Pres_plat","Glucose"],
    },
    '9 Variables': {
        1: ["VR","Covid","DM", "Time","Creatinine","HAS", "VT","DHL","Glucose"],
        2: ["VR","Covid","Time", "Creatinine","HAS","VT", "DHL","Pres_plat","Glucose"],
        3: ["Creatinine","VR","Glucose", "Time","DHL","Age", "Platelets","PEEP","Covid"],
        4: ["VR","Glucose","Creatinine", "Time","Covid","DHL", "DP","HAS","VT"],
        5: ["VR","Covid","DM", "Time","HAS","VT", "DHL","Pres_plat","Glucose"],
    },
    '10 Variables': {
        1: ["VR","Covid","DM", "Time","Creatinine","HAS", "VT","DHL","Pres_plat", "Glucose"],
        2: ["VR","Glucose","Creatinine", "Pres_plat","Time","Covid", "DHL","DP","HAS","VT"],
        3: ["Creatinine","VR","Glucose", "Time","DHL","Age", "Platelets","PEEP","Covid", "BMI"],
        4: ["VR","Creatinine","Pres_plat", "Glucose","Covid","VT", "Time","DP","Age","HAS"],
        5: ["VR","DHL","Covid", "Pres_plat","Creatinine","Glucose", "HAS","Vasopressor","FC","Age"],        
    },
}

# Function to calculate mortality probability
def calculate_mortality(table, model_number, input_variables):
    if table not in model_coefficients or model_number not in model_coefficients[table]:
        raise ValueError(f"Model {model_number} for {table} is not available.")
    coefficients = model_coefficients[table][model_number]
    if len(input_variables) != len(coefficients) - 1:
        raise ValueError(f"Model {model_number} expects {len(coefficients)-1} input variables.")
    model = LogisticModel(coefficients)
    return model.predict_mortality_probability(input_variables)

# Function to handle the button click and calculate the probability
def on_calculate():
    try:
        # Get the selected table and model number
        table_name = table_dropdown.get()
        model_number = int(model_dropdown.get())

        # Get input variable values from the text fields
        input_values = []
        for entry in input_entries:
            value = float(entry.get())
            input_values.append(value)

        # Calculate mortality probability
        probability = calculate_mortality(table_name, model_number, input_values)

        # Display the result in a message box
        messagebox.showinfo("Mortality Probability", f"Mortality Probability: {probability:.4f}")

    except Exception as e:
        messagebox.showerror("Error", str(e))

# Create the main application window
root = tk.Tk()
root.title("Mortality Probability Calculator (Tables VII to XV)")

# Dropdown menu for table selection (Table VII to Table XV)
table_label = ttk.Label(root, text="# Variables:")
table_label.grid(column=0, row=0, padx=10, pady=10)
table_dropdown = ttk.Combobox(root, values=list(model_coefficients.keys()))
table_dropdown.grid(column=1, row=0, padx=10, pady=10)
table_dropdown.current(0)

# Dropdown menu for model selection within the chosen table
model_label = ttk.Label(root, text="Select Model:")
model_label.grid(column=0, row=1, padx=10, pady=10)
model_dropdown = ttk.Combobox(root, values=[])
model_dropdown.grid(column=1, row=1, padx=10, pady=10)

# Input fields for variables (this will be dynamically updated based on the selected model)
input_labels = []
input_entries = []

# Function to update input fields based on selected table and model
def update_input_fields(event):
    # Get the selected table and model number
    table_name = table_dropdown.get()
    model_number = int(model_dropdown.get())

    # Clear existing input fields
    for label in input_labels:
        label.destroy()
    for entry in input_entries:
        entry.destroy()
    
    input_labels.clear()
    input_entries.clear()

    # Get the variable names for the selected model
    variable_names = model_variable_names[table_name][model_number]

    # Create input fields dynamically with proper variable names
    for i, var_name in enumerate(variable_names):
        input_label = ttk.Label(root, text=f"{var_name}:")
        input_label.grid(column=0, row=i + 2, padx=10, pady=5)
        input_entry = ttk.Entry(root)
        input_entry.grid(column=1, row=i + 2, padx=10, pady=5)
        input_labels.append(input_label)
        input_entries.append(input_entry)

# Bind the dropdown selection change to update the model options and input fields
def update_model_options(event):
    table_name = table_dropdown.get()
    model_dropdown['values'] = list(model_coefficients[table_name].keys())
    model_dropdown.current(0)
    update_input_fields(None)

table_dropdown.bind("<<ComboboxSelected>>", update_model_options)
model_dropdown.bind("<<ComboboxSelected>>", update_input_fields)

# Button to calculate the probability
calculate_button = ttk.Button(root, text="Calculate Probability", command=on_calculate)
calculate_button.grid(column=0, row=12, columnspan=2, padx=10, pady=10)

# Start the application loop
root.mainloop()


In [12]:
pip install pyinstaller

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.3/703.3 KB 965.1 kB/s eta 0:00:001.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.8/322.8 KB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.
